In [1]:
# import all libraries

from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 120)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('../data/train')
valid_files, valid_targets = load_dataset('../data/validation')
test_files, test_targets = load_dataset('../data/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("../data/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 120 total dog categories.
There are 20580 total dog images.

There are 14531 training dog images.
There are 4232 validation dog images.
There are 1817 test dog images.


In [4]:
from keras.applications import ResNet50
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from keras.models import Model, Sequential

pretrained_model= ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3)) 

for layer in pretrained_model.layers:
    layer.trainable = False
    
x = pretrained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
#dropout
predictions = Dense(120, activation='softmax')(x)

model = Model(inputs = pretrained_model.input, outputs=predictions) 

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
# training image augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from keras.callbacks import History 
from keras.applications import vgg16

batch_size = 16

# this is the augmentation configuration I will use for training
train_datagen = ImageDataGenerator(rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   preprocessing_function=vgg16.preprocess_input)

# This is the augmentation configuration I will use for testing/validation... just a rescale
test_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)

# This is the generator which will read pictures found in my training subset
train_generator = train_datagen.flow_from_directory('../data/train/',
                                                    target_size = (224, 224),
                                                    batch_size = batch_size,
                                                    shuffle=True,
                                                    class_mode = 'categorical',
                                                    seed=42)

# This is the generator for validation data
validation_generator = test_datagen.flow_from_directory('../data/validation/',
                                                        target_size = (224, 224),
                                                        batch_size = batch_size,
                                                        class_mode = 'categorical')

Found 14531 images belonging to 120 classes.
Found 4232 images belonging to 120 classes.


In [6]:
from keras.callbacks import ModelCheckpoint, History

history = History()
checkpoint_augmentation = '../data/checkpoint_augmentation/resnet'
model_checkpoint_augmentation = ModelCheckpoint(filepath=checkpoint_augmentation,
                                                verbose=1,
                                                save_best_only=True)

In [9]:
%%time
model.fit_generator(train_generator,
                    epochs = 10,
                    validation_data = validation_generator,
                    #callbacks = [model_checkpoint_augmentation, history],
                    verbose=2)

Epoch 1/10
 - 2117s - loss: 1.2043 - accuracy: 0.6457 - val_loss: 0.2561 - val_accuracy: 0.6935
Epoch 2/10
 - 2015s - loss: 1.0749 - accuracy: 0.6757 - val_loss: 1.4768 - val_accuracy: 0.7049
Epoch 3/10
 - 2095s - loss: 0.9691 - accuracy: 0.7064 - val_loss: 0.3458 - val_accuracy: 0.6909
Epoch 4/10
 - 2071s - loss: 0.9048 - accuracy: 0.7258 - val_loss: 2.6876 - val_accuracy: 0.7004
Epoch 5/10
 - 2083s - loss: 0.8494 - accuracy: 0.7406 - val_loss: 0.6907 - val_accuracy: 0.7030
Epoch 6/10
 - 2096s - loss: 0.8205 - accuracy: 0.7486 - val_loss: 0.4392 - val_accuracy: 0.7046
Epoch 7/10


KeyboardInterrupt: 

In [8]:
1+1

2